# Stability test usage example

In this notebook, a demonstration of how to perform a stability test is provided. As test case, a problem from [Whitson and Brulé SPE Monograph](https://books.google.com.br/books/about/Phase_Behavior.html?id=Z4cQAQAAMAAJ&redir_esc=y). More exactly, a test for phase split is check for the Problem 18 provided in the appendix. The mixture of the problem is defined as:

| Component | molar fraction |
|-----------|---------------:|
| $C_1$     |            0.5 |
| $C_4$     |           0.42 |
| $C_{10}$  |           0.08 |

Critical pressures and temperatures can be gathered easily elsewhere.

* Importing the necessary libs:

In [1]:
import attr
import numpy as np
import warnings

from gibbs.models.ceos import PengRobinson78
from gibbs.stability_analysis import stability_test

warnings.filterwarnings('ignore')

* Defining a class problem for example 18 from Whitson with Peng Robinson cubic EoS from available models:

In [2]:
@attr.s(auto_attribs=True)
class WhitsonModel:
    z: np.ndarray
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray

    @property
    def model(self):
        return PengRobinson78(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor, 
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

* Instantiating the class problem at a known overall composition and BIPs:

In [3]:
z = np.array([0.5, 0.42, 0.08])
omegas = np.array([0.0115, 0.1928, 0.4902])
Tcs = np.array([190.556, 425.16667, 617.666667])
Pcs = np.array([4604318.9, 3796942.8, 2.096e6])
kijs = np.zeros((3, 3))
model = WhitsonModel(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

* Performing a stability test for the model at known PT-condition:

In [4]:
P = 3.447e6
T = 410.928

result = stability_test(model, P, T, z)

print(result.phase_split)

True
